# Matplotlib Examples

This notebook demonstrates using matplotlib with nbappinator.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import nbappinator as nb

## 1. Basic Matplotlib in App

In [ ]:
def draw_chart(app):
    with app.messages:
        app.status("Drawing chart...")

        # Get parameters from app
        chart_type = app["chart_type"]
        n_points = app["n_points"]

        # Generate data
        np.random.seed(42)
        x = np.linspace(0, 10, n_points)
        y = np.sin(x) + np.random.normal(0, 0.2, n_points)

        # Create figure
        fig, ax = plt.subplots(figsize=(10, 6))

        if chart_type == "Line":
            ax.plot(x, y, "b-", linewidth=2, label="sin(x) + noise")
            ax.plot(x, np.sin(x), "r--", linewidth=1, label="sin(x)")
        elif chart_type == "Scatter":
            ax.scatter(x, y, c=y, cmap="viridis", s=50, alpha=0.7)
        elif chart_type == "Bar":
            ax.bar(x[:20], y[:20], width=0.4, color="steelblue", alpha=0.8)
        elif chart_type == "Area":
            ax.fill_between(x, y, alpha=0.5, color="steelblue")
            ax.plot(x, y, "b-", linewidth=1)

        ax.set_xlabel("X")
        ax.set_ylabel("Y")
        ax.set_title(f"{chart_type} Chart ({n_points} points)")
        ax.legend()
        ax.grid(True, alpha=0.3)

        # Display in app
        app.tab(0).clear().matplotlib(fig)
        plt.close(fig)  # Close to free memory

        app.done("Chart drawn")


app = nb.App(tabs=["Chart"], footer="Messages", header="Config")

app.config.select("chart_type", options=["Line", "Scatter", "Bar", "Area"], default="Line", label="Chart Type")
app.config.slider("n_points", min_val=10, max_val=200, default=100, label="Points")
app.config.button("draw", on_click=draw_chart, label="Draw Chart", status=True)

draw_chart(app)

app.display()

## 2. Multiple Subplots

In [ ]:
def draw_subplots(app):
    with app.messages:
        app.status("Drawing subplots...")

        np.random.seed(42)

        fig, axes = plt.subplots(2, 2, figsize=(12, 10))

        # Line plot
        x = np.linspace(0, 10, 100)
        axes[0, 0].plot(x, np.sin(x), "b-", label="sin")
        axes[0, 0].plot(x, np.cos(x), "r-", label="cos")
        axes[0, 0].set_title("Trigonometric Functions")
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)

        # Histogram
        data = np.random.randn(1000)
        axes[0, 1].hist(data, bins=30, color="steelblue", edgecolor="white", alpha=0.8)
        axes[0, 1].set_title("Normal Distribution")
        axes[0, 1].set_xlabel("Value")
        axes[0, 1].set_ylabel("Frequency")

        # Scatter plot
        x = np.random.randn(100)
        y = x + np.random.randn(100) * 0.5
        axes[1, 0].scatter(x, y, c=y, cmap="coolwarm", s=50, alpha=0.7)
        axes[1, 0].set_title("Scatter with Color")
        axes[1, 0].set_xlabel("X")
        axes[1, 0].set_ylabel("Y")

        # Bar chart
        categories = ["A", "B", "C", "D", "E"]
        values = np.random.randint(10, 100, 5)
        colors = plt.cm.viridis(np.linspace(0.2, 0.8, 5))
        axes[1, 1].bar(categories, values, color=colors)
        axes[1, 1].set_title("Category Comparison")
        axes[1, 1].set_ylabel("Value")

        plt.tight_layout()

        app.tab(0).clear().matplotlib(fig)
        plt.close(fig)

        app.done("Subplots drawn")


app2 = nb.App(tabs=["Subplots"], footer="Messages", header="Config")
app2.config.button("draw", on_click=draw_subplots, label="Draw Subplots", status=True)

draw_subplots(app2)

app2.display()

## 3. Time Series

In [ ]:
import pandas as pd


def draw_timeseries(app):
    with app.messages:
        app.status("Drawing time series...")

        # Generate time series data
        np.random.seed(42)
        dates = pd.date_range("2024-01-01", periods=252, freq="B")  # Business days

        # Simulate stock prices
        returns = np.random.randn(252) * 0.02
        price = 100 * np.cumprod(1 + returns)

        # Moving averages
        ma20 = pd.Series(price).rolling(20).mean()
        ma50 = pd.Series(price).rolling(50).mean()

        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), height_ratios=[3, 1])

        # Price chart
        ax1.plot(dates, price, "b-", linewidth=1, label="Price")
        ax1.plot(dates, ma20, "orange", linewidth=1.5, label="20-day MA")
        ax1.plot(dates, ma50, "red", linewidth=1.5, label="50-day MA")
        ax1.fill_between(dates, price, alpha=0.1)
        ax1.set_ylabel("Price ($)")
        ax1.set_title("Stock Price with Moving Averages")
        ax1.legend(loc="upper left")
        ax1.grid(True, alpha=0.3)

        # Volume chart (simulated)
        volume = np.random.randint(1_000_000, 5_000_000, 252)
        colors = ["green" if r > 0 else "red" for r in returns]
        ax2.bar(dates, volume, color=colors, alpha=0.7, width=1)
        ax2.set_ylabel("Volume")
        ax2.set_xlabel("Date")
        ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f"{x / 1e6:.1f}M"))

        plt.tight_layout()

        app.tab(0).clear().matplotlib(fig)
        plt.close(fig)

        app.done("Time series drawn")


app3 = nb.App(tabs=["Time Series"], footer="Messages", header="Config")
app3.config.button("draw", on_click=draw_timeseries, label="Draw Time Series", status=True)

draw_timeseries(app3)

app3.display()

## 4. Heatmap

In [ ]:
def draw_heatmap(app):
    with app.messages:
        app.status("Drawing heatmap...")

        # Generate correlation matrix
        np.random.seed(42)
        n_assets = 10
        labels = [f"Asset {i + 1}" for i in range(n_assets)]

        # Create a random positive semi-definite correlation matrix
        random_data = np.random.randn(n_assets, n_assets)
        corr = np.corrcoef(random_data)

        fig, ax = plt.subplots(figsize=(10, 8))

        im = ax.imshow(corr, cmap="RdYlGn", vmin=-1, vmax=1)

        # Add colorbar
        cbar = ax.figure.colorbar(im, ax=ax)
        cbar.ax.set_ylabel("Correlation", rotation=-90, va="bottom")

        # Labels
        ax.set_xticks(np.arange(n_assets))
        ax.set_yticks(np.arange(n_assets))
        ax.set_xticklabels(labels, rotation=45, ha="right")
        ax.set_yticklabels(labels)

        # Add correlation values
        for i in range(n_assets):
            for j in range(n_assets):
                ax.text(j, i, f"{corr[i, j]:.2f}", ha="center", va="center", color="black", fontsize=8)

        ax.set_title("Asset Correlation Matrix")
        plt.tight_layout()

        app.tab(0).clear().matplotlib(fig)
        plt.close(fig)

        app.done("Heatmap drawn")


app4 = nb.App(tabs=["Heatmap"], footer="Messages", header="Config")
app4.config.button("draw", on_click=draw_heatmap, label="Draw Heatmap", status=True)

draw_heatmap(app4)

app4.display()